In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd
import polars as pl
import pyreadr
import seaborn as sns
import urllib.request

In [ ]:
def download_dataset(file_url: str, file_directory: str, file_name: str) -> str:
    directory = os.path.join(os.path.abspath(os.curdir), file_directory)
    if not os.path.exists(directory):
        os.makedirs(directory)
    
    file = os.path.join(directory, file_name)
    if not os.path.isfile(file):
        urllib.request.urlretrieve(file_url, file)
    
    return file

def convert_dataset(original_path: str, destination_path: str) -> str:
    if not os.path.isfile(destination_path):
        result = pyreadr.read_r(original_path)
        df = result['df']

        df.to_csv(destination_path, index=True)
    
    return destination_path

In [ ]:
DATASET_URL = 'https://github.com/yaleemmlc/admissionprediction/raw/master/Results/5v_cleandf.RData'
FILE_PATH = 'Datasets'
FILE_NAME = 'Results.rdata'

# Downloads the Original Dataset (RData File)
DATASET_RDATA_PATH = download_dataset(DATASET_URL, FILE_PATH, FILE_NAME)

# Converts the RData File to a New CSV File
DATASET_CSV_PATH = convert_dataset(DATASET_RDATA_PATH, DATASET_RDATA_PATH.replace('.rdata', '.csv'))

In [ ]:
df = pl.read_csv(DATASET_CSV_PATH)

df.head()

In [ ]:
df.describe()

In [ ]:
# Elimina las columnas donde todos sus elementos sean iguales
columns_to_remove = df.columns[df.n_unique() == 1]
df_witout_nunique = df.drop(columns=columns_to_remove)
df_witout_nunique.head()

In [ ]:
significantColumns = ['cc_chestpain', 'cc_breathingdifficulty', 'cc_syncope', 'cc_unresponsive', 'cc_seizure-newonset', 'cc_seizure-priorhxof', 'cc_seizures', 'cc_bleeding/bruising', 'cc_hyperglycemia', 'cc_hypertension', 'cc_hypotension', 'cc_strokealert', 'cc_overdose-accidental', 'cc_overdose-intentional', 'cc_suicidal']

In [ ]:
# If any of the significant columns are on true, automatically has the flag of emergency
df_witout_nunique = df_witout_nunique.to_pandas()
df_witout_nunique['emergency_flag_column'] = df_witout_nunique[significantColumns].any(axis=1)

print(df_witout_nunique['emergency_flag_column'].value_counts())

In [ ]:
from sklearn.preprocessing import LabelEncoder
df_converted = df_witout_nunique.copy()

# Inicializar el codificador de etiquetas
label_encoder = LabelEncoder()

# Identificar las variables categóricas
categorical_columns = []
for column in df_converted.columns:
    # Verificar si la columna es de tipo 'object' o 'category'
    if df_converted[column].dtype == 'object' or df_converted[column].dtype == 'category' :  
        categorical_columns.append(column)

# Convertir las variables categóricas usando LabelEncoder
for column in categorical_columns:
    df_converted[column] = label_encoder.fit_transform(df_converted[column])

print(df_converted['gender'].value_counts())


In [ ]:
columnsWithNan = df_converted.isna().any().pipe(lambda x: x.index[x])
print(len(columnsWithNan))

# Reemplazo los valores nulos con -999. Se podría intentar en una proxima iteracion con algo como col.mode()[0] o valores significativos reales
df_filled = df_converted.apply(lambda col: col.fillna(-999))

newColumnsWithNan = df_filled.isna().any().pipe(lambda x: x.index[x])
print(len(newColumnsWithNan))


In [ ]:
# Entrenamiento del modelo
from sklearn.ensemble import RandomForestClassifier
# División de datos
X = df_filled.drop(columns=['emergency_flag_column'])
y = df_filled['emergency_flag_column']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
# Validación del modelo
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')

In [ ]:
from sklearn.metrics import confusion_matrix

confusion = confusion_matrix(y_test, y_pred)
TP = confusion[1, 1]
FP = confusion[0, 1]
TN = confusion[0, 0]
FN = confusion[1, 0]

print(f"Verdaderos Positivos: {str(TP)}")

In [ ]:
# Codificación de variables categóricas
df_encoded = pd.get_dummies(df_witout_nunique)

In [ ]:
from sklearn.impute import SimpleImputer

# Manejo de valores nulos
imputer = SimpleImputer(strategy='most_frequent')  
df_without_null = imputer.fit_transform(df_witout_nunique)
print(df_without_null)

In [ ]:
# Crear un DataFrame con las características y las etiquetas codificadas
df_plot = X.copy()
df_plot['emergency_flag'] = y

# Graficar la distribución de las clases
plt.figure(figsize=(8, 6))
sns.countplot(x='emergency_flag', data=df_plot)
plt.title('Distribución de Clases')
plt.xlabel('Clase')
plt.ylabel('Conteo')
plt.show()